 Neste notebook implementei um algoritmo simples de segmentação utilizado em [Frog classification using machine learning techniques](https://www.sciencedirect.com/science/article/abs/pii/S0957417408001504). É um algoritmo simples mas que pode ser considerado uma boa heuristica para evitar selecionar partes sem cantos.

# Importando Libs

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import matplotlib.pyplot as plt
import os
import cv2
import IPython.display as ipd
from IPython.display import Audio, IFrame, display
import plotly.graph_objects as go
import librosa
import librosa.display


import warnings
warnings.filterwarnings('ignore')

# O algoritmo
O algoritmo de segmentação basicamente consiste na ideia de selecionar e retirar a parte do audio que possui a maior amplitude. Depois fazer a mesma coisa para o restante do audio, e repetir até que o audio restante não ultrapasse uma amplitude pre-determinada.

In [ ]:
# The Function return the segments extracted from audio and the rest of audio
def segmentation(x,tr):
    
    resp = [];
    
    while(len(x) >= (sr*2)):
        if(max(x) < tr):
            break
            
        time_amplitude_max = np.argmax(x)
        
        #Higher amplitude is before 1s 
        if(time_amplitude_max < ((1)*sr)):
            resp.append(x[:2*sr])
            x = x[2*sr:]
            
        #Higher amplitude is on the last 1s     
        elif(time_amplitude_max > (len(x) - ((1)*sr))):
            resp.append(x[-(2*sr):])
            x = x[:-2*sr]
            
        else:
            resp.append(x[time_amplitude_max-int((1)*sr):time_amplitude_max+int((1)*sr)])
            x = np.concatenate((x[:time_amplitude_max-int((1)*sr)-1],x[time_amplitude_max+int((1)*sr)+1:]))
            
    return resp,x     

# Exemplo 1
Esse primeiro exemplo é muito bom pois temos um audio que é meio dividido entre partes de silencio e canto, esse passaro foi legal.

In [ ]:
x , sr = librosa.load("../input/birdsong-recognition/train_audio/cacwre/XC132895.mp3")
librosa.display.waveplot(x, sr=sr)
Audio(x, rate=sr)

Aqui aplicamos o algoritmo ao aúdio, com o threshold = 0.25

In [ ]:
segments,resto = segmentation(x,0.25)
print("Foram recuperados {} Segmentos".format(len(segments)));

Em baixo temos os pedaço do audio que não foi selecionado

In [ ]:
librosa.display.waveplot(resto, sr=sr)
Audio(resto, rate=sr)

O segmento com maior amplitude.

In [ ]:
librosa.display.waveplot(segments[1], sr=sr)
Audio(segments[1], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[1],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

# Exemplo 2

In [ ]:
x , sr = librosa.load("../input/birdsong-recognition/train_audio/btywar/XC133590.mp3")
librosa.display.waveplot(x, sr=sr)
Audio(x, rate=sr)

Aqui foi recuperado somente 1 Segmento(Sem canto), mesmo havendo outras partes contendo um canto

In [ ]:
segments,resto = segmentation(x,0.25)
print("Foram recuperados {} Segmentos".format(len(segments)));

In [ ]:
librosa.display.waveplot(segments[0], sr=sr)
Audio(segments[0], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[0],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

# Exemplo 3

In [ ]:
x , sr = librosa.load("../input/birdsong-recognition/train_audio/killde/XC143721.mp3")
librosa.display.waveplot(x, sr=sr)
Audio(x, rate=sr)

In [ ]:
segments,resto = segmentation(x,0.25)
print("Foram recuperados {} Segmentos".format(len(segments)));

In [ ]:
librosa.display.waveplot(segments[0], sr=sr)
Audio(segments[0], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[0],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

# Vamos testar os exemplos acima normalizando antes da segmentação

In [ ]:
def normaliza(x):
    mi = x.mean()
    sigma = np.std(x)
    
    x = x-mi;
    return (x/sigma) 

## Exemplo 1

In [ ]:
x , sr = librosa.load("../input/birdsong-recognition/train_audio/cacwre/XC132895.mp3")
x = normaliza(x)               
librosa.display.waveplot(x, sr=sr)
Audio(x, rate=sr)

Aparentemente para threshold = 1, são selecionados muitos segmentos.

In [ ]:
segments,resto = segmentation(x,1)
print("Foram recuperados {} Segmentos".format(len(segments)));

Alguns sem canto.

In [ ]:
librosa.display.waveplot(segments[51], sr=sr)
Audio(segments[51], rate=sr)

Achei 5 melhor

In [ ]:
segments,resto = segmentation(x,5)
print("Foram recuperados {} Segmentos".format(len(segments)));

In [ ]:
librosa.display.waveplot(segments[21], sr=sr)
Audio(segments[21], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[0],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

## Exemplo 2

Esse caso 2 é interessante, porque ele ta meio baixo, e tem um momento que tem um ruido perto do microfone. Normalizando a gente consegue selecionar mais cantos, apesar ainda de vir algumas coisas como uma buzina.

In [ ]:
x , sr = librosa.load("../input/birdsong-recognition/train_audio/btywar/XC133590.mp3")
x = normaliza(x)
librosa.display.waveplot(x, sr=sr)
Audio(x, rate=sr)

In [ ]:
segments,resto = segmentation(x,5)
print("Foram recuperados {} Segmentos".format(len(segments)));

O primeiro segmento escolhido não tem nada.

In [ ]:
librosa.display.waveplot(segments[0], sr=sr)
Audio(segments[0], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[0],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
librosa.display.waveplot(segments[2], sr=sr)
Audio(segments[2], rate=sr)

No entanto nestes dois audios/spectrogramas abaixo a gente consegue pegar perfeitamente o canto.

In [ ]:
melspec = librosa.feature.melspectrogram(segments[2],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
librosa.display.waveplot(segments[4], sr=sr)
Audio(segments[4], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[4],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

## Exemplo 3

Esse é mais Trivial

In [ ]:
x , sr = librosa.load("../input/birdsong-recognition/train_audio/killde/XC143721.mp3")
x = normaliza(x)
librosa.display.waveplot(x, sr=sr)
Audio(x, rate=sr)

In [ ]:
segments,resto = segmentation(x,5)
print("Foram recuperados {} Segmentos".format(len(segments)));

In [ ]:
librosa.display.waveplot(segments[0], sr=sr)
Audio(segments[0], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[0],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
librosa.display.waveplot(segments[1], sr=sr)
Audio(segments[1], rate=sr)

In [ ]:
melspec = librosa.feature.melspectrogram(segments[1],sr=sr)
melspec = librosa.power_to_db(melspec).astype(np.float32)
plt.figure(figsize=(10, 4))
librosa.display.specshow(melspec, x_axis='time',y_axis='mel', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-Spectrogram')
plt.tight_layout()
plt.show()